In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.factor_engineer import FactorEngineer, GroupFactorEngineer
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'


In [3]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)

In [ ]:
(-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))

In [8]:
stocks_df[stocks_df.ticker == 'AAPL'].index.max()

Timestamp('2022-03-16 00:00:00')

In [5]:
stacked_hist = stocks_df.copy()

In [4]:
stacked_hist = FinancialDF(stacked_hist)

In [15]:
type(stacked_hist.ticker)

__main__.CandlesSeries

In [7]:
s_time_chunk = time.time()
# stacked_hist['a1'] = stacked_hist.calc_alpha(alpha101.alpha1, n_std=20, n_ts_arg_max=5)
stacked_hist['a1'] = alpha101.alpha1(stacked_hist, n_std=20, n_ts_arg_max=5)
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 58.95735692977905 sec


In [21]:
type(pd.concat([stocks_df, stacked_hist]))

pandas.core.frame.DataFrame

In [ ]:
stacked_hist['alpha1'] =

In [13]:
n_delta = 2

# stacked_hist[f'volume_delta{n_delta}'] = stacked_hist.groupby('ticker').volume.apply(ic.delta_volume, n=n_delta)
# stacked_hist['ret_intraday'] = stacked_hist.closeadj / stacked_hist.openadj - 1

stacked_hist[f'volume_delta{n_delta}'] = stacked_hist.engineer(ic.delta_volume, 'volume', n=n_delta)
stacked_hist['ret_intraday'] = ic.pct_change(stacked_hist.closeadj, stacked_hist.openadj)

stacked_hist['rank_volumedelta'] = stacked_hist.cross_pctrank(f'volume_delta{n_delta}')
stacked_hist['rank_ret_intraday'] = stacked_hist.cross_pctrank('ret_intraday')

n_roll = 6
s_time_chunk = time.time()
tickers = stacked_hist.ticker.unique()
corr_stack = []
n_tickers = len(tickers)
for i, ticker in enumerate(tickers):
    temp_hist = stacked_hist[stacked_hist.ticker == ticker]
    corr_stack.append(temp_hist['rank_volumedelta'].rolling(n_roll).corr(temp_hist['rank_ret_intraday']))

    if (i+1) % 50 == 0:
        print(f'Iteration progress: {round((i+1)/n_tickers * 100, 2)} %')
stacked_hist['corr_ranks'] = pd.concat(corr_stack)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")
stacked_hist['alpha'] = -1 * stacked_hist.corr_ranks


Iteration progress: 0.48 %
Iteration progress: 0.97 %
Iteration progress: 1.45 %
Iteration progress: 1.93 %
Iteration progress: 2.41 %
Iteration progress: 2.9 %
Iteration progress: 3.38 %
Iteration progress: 3.86 %
Iteration progress: 4.34 %
Iteration progress: 4.83 %
Iteration progress: 5.31 %
Iteration progress: 5.79 %
Iteration progress: 6.28 %
Iteration progress: 6.76 %
Iteration progress: 7.24 %
Iteration progress: 7.72 %
Iteration progress: 8.21 %
Iteration progress: 8.69 %
Iteration progress: 9.17 %
Iteration progress: 9.66 %
Iteration progress: 10.14 %
Iteration progress: 10.62 %
Iteration progress: 11.1 %
Iteration progress: 11.59 %
Iteration progress: 12.07 %
Iteration progress: 12.55 %
Iteration progress: 13.03 %
Iteration progress: 13.52 %
Iteration progress: 14.0 %
Iteration progress: 14.48 %
Iteration progress: 14.97 %
Iteration progress: 15.45 %
Iteration progress: 15.93 %
Iteration progress: 16.41 %
Iteration progress: 16.9 %
Iteration progress: 17.38 %
Iteration progre

In [22]:
4724/60

78.73333333333333

In [7]:
A = stacked_hist[stacked_hist.ticker == 'A']

In [9]:
dummy_hist = stacked_hist[stacked_hist.ticker.isin(stacked_hist.ticker.unique()[:100])]

In [11]:
n_roll = 6

s_time_chunk = time.time()
stack = []
for ticker in dummy_hist.ticker.unique():
    temp_hist = dummy_hist[dummy_hist.ticker == ticker]
    stack.append(temp_hist['rank_volumedelta'].rolling(n_roll).corr(temp_hist['rank_ret_intraday']))
dummy_hist['corr_ranks'] = pd.concat(stack).values
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 0.557974100112915 sec


/var/folders/wq/0z599tw14bdgq8gkgc7wjtlc0000gn/T/ipykernel_29506/1976834104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_hist['corr_ranks'] = pd.concat(stack).values


In [12]:
dummy_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,volume_delta2,ret_intraday,rank_volumedelta,rank_ret_intraday,corr_ranks
date,,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,0.017857,NaN,0.811883,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,NaN,0.001107,NaN,0.573025,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.498527,0.031215,0.822699,0.874418,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.012721,0.008358,0.488345,0.778275,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,-0.434874,0.020202,0.264791,0.863145,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07,ACER,2.53,2.530,2.380,2.46,2.530000,2.530000,2.380000,2.460,71484.0,1.056374,-0.027668,0.831209,0.338738,0.368989
2022-03-08,ACER,2.46,2.490,2.400,2.42,2.460000,2.490000,2.400000,2.420,61114.0,0.689521,-0.016260,0.753617,0.200117,-0.384314
2022-03-09,ACER,2.44,2.540,2.420,2.54,2.440000,2.540000,2.420000,2.540,66971.0,-0.065214,0.040984,0.596810,0.856932,-0.420014


In [20]:
stack = []
for ticker in dummy_hist.ticker.unique():
    temp_hist = dummy_hist[dummy_hist.ticker == ticker]
    stack.append(temp_hist['rank_volumedelta'].rolling(n_roll).corr(temp_hist['rank_ret_intraday']))


In [21]:
pd.concat(stack)

date
2012-01-03         NaN
2012-01-04         NaN
2012-01-05         NaN
2012-01-06         NaN
2012-01-09         NaN
                ...   
2022-03-07    0.368989
2022-03-08   -0.384314
2022-03-09   -0.420014
2022-03-10   -0.744048
2022-03-11   -0.441385
Length: 122132, dtype: float64

In [19]:
dummy_hist[dummy_hist.ticker == 'ABAT']

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,volume_delta2,ret_intraday,rank_volumedelta,rank_ret_intraday
date,,,,,,,,,,,,,,
2012-01-03,ABAT,0.450,0.480,0.430,0.450,0.450,0.480,0.430,0.450,497400.0,NaN,0.000000,NaN,0.532129
2012-01-04,ABAT,0.430,0.460,0.420,0.460,0.430,0.460,0.420,0.460,508200.0,NaN,0.069767,NaN,0.967399
2012-01-05,ABAT,0.460,0.460,0.410,0.420,0.460,0.460,0.410,0.420,402400.0,-0.211948,-0.086957,0.471384,0.010248
2012-01-06,ABAT,0.440,0.440,0.410,0.420,0.440,0.440,0.410,0.420,351500.0,-0.368665,-0.045455,0.277972,0.033818
2012-01-09,ABAT,0.420,0.420,0.390,0.400,0.420,0.420,0.390,0.400,408600.0,0.015290,-0.047619,0.529583,0.031302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-23,ABAT,0.050,0.058,0.050,0.050,0.050,0.058,0.050,0.050,220720.0,0.935595,0.000000,0.914532,0.309864
2015-12-24,ABAT,0.050,0.050,0.050,0.050,0.050,0.050,0.050,0.050,14530.0,-0.717219,0.000000,0.612822,0.447279
2015-12-28,ABAT,0.031,0.050,0.031,0.042,0.031,0.050,0.031,0.042,151778.0,-0.374476,0.354839,0.340552,0.998491


In [44]:
# stocks_hist['alpha'] = stocks_hist.groupby('date').tsargmax_withcond.rank(pct=True).values

In [56]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,closeadj_sqr,closeadj_argmax,retvol20_sqr,retvol20_argmax,tsargmax_withcond,alpha
date,,,,,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,579.557476,NaN,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,570.349924,NaN,NaN,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,596.189889,NaN,NaN,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,609.151761,NaN,NaN,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,641.507584,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0,0.000040,3.0,4.0,0.8
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0,0.000040,2.0,5.0,1.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0,0.000040,1.0,4.0,0.8


In [27]:
s_time_chunk = time.time()
stocks_hist['closeadj_argmax'] = stocks_hist.groupby('ticker').closeadj.rolling(5).apply(np.argmax, engine='cython', raw=True).values
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 18.187129259109497 sec


In [28]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,op1,op2
date,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0
